In [1]:
!pip install dlib

In [2]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=6b5dec8b204dfe2256c784f1209f96d1aa25cbaf64b92893e55f7628dfe3652a
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import os
import cv2
import numpy as np
import face_recognition
from sklearn.cluster import DBSCAN
from collections import defaultdict

# Step 1: Read images from directories
def get_image_paths(directory_list):
    image_paths = []
    for directory in directory_list:
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.endswith(".jpg") or file.endswith(".png"):
                    image_paths.append(os.path.join(root, file))
    return image_paths

# Step 2 & 3: Detect faces and extract embeddings, modified to include directory information
def get_face_embeddings(image_paths):
    face_embeddings = []
    image_info = []  # Modified to store a tuple of image path and directory
    for image_path in image_paths:
        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.face_locations(image, model="cnn")
        if face_locations:
            # Assuming one face per image for simplicity
            face_encoding = face_recognition.face_encodings(image, known_face_locations=face_locations)[0]
            face_embeddings.append(face_encoding)
            image_info.append((image_path, os.path.dirname(image_path)))  # Store both image name and directory
    return face_embeddings, image_info

# Step 4: Clustering embeddings
def cluster_faces(face_embeddings):
    clustering_model = DBSCAN(eps=0.5, min_samples=2, metric="euclidean").fit(face_embeddings)
    labels = clustering_model.labels_
    return labels

# Step 5: Group image information by cluster, modified to handle directory information
def group_images_by_cluster(labels, image_info):
    clusters = defaultdict(list)
    for label, (image_name, directory) in zip(labels, image_info):
        clusters[label].append((image_name, directory))  # Store tuples of image name and directory in clusters
    return clusters

# Example usage
directory_list = ["/content/group1", "/content/group2"]  # Update these paths
image_paths = get_image_paths(directory_list)
face_embeddings, image_info = get_face_embeddings(image_paths)
labels = cluster_faces(face_embeddings)
clusters = group_images_by_cluster(labels, image_info)

output_file = "clusters.txt"
with open(output_file, "w") as file:
    for label, image_data in clusters.items():
        if label != -1:  # -1 is for outliers
            file.write(f"Cluster {label}:\n")
            for image_name, directory in image_data:
                file.write(f"{image_name} (Directory: {directory})\n")
print("Clusters saved to:", output_file)

# Print out clusters of images with similar faces along with their directories
# for label, image_data in clusters.items():
#     if label != -1:  # -1 is for outliers
#         print(f"Cluster {label}:")
#         for image_name, directory in image_data:
#             print(f"{image_name} (Directory: {directory})")

In [ ]:
def add_file_to_clusters(file_path, clusters):
    # Load the new image and extract its face embeddings
    image = face_recognition.load_image_file(file_path)
    face_locations = face_recognition.face_locations(image, model="cnn")
    if face_locations:
        # Assuming one face per image for simplicity
        face_encoding = face_recognition.face_encodings(image, known_face_locations=face_locations)[0]
        # Compare the face embedding with embeddings in each cluster and add it to the most suitable cluster
        for cluster_label, cluster_data in clusters.items():
            for existing_file_path, _ in cluster_data:
                existing_image = face_recognition.load_image_file(existing_file_path)
                existing_face_encoding = face_recognition.face_encodings(existing_image)[0]
                # Compare the similarity between the new face and the existing faces in the cluster
                face_distance = face_recognition.face_distance([existing_face_encoding], face_encoding)[0]
                # If the face is similar enough, add the new file to the cluster
                if face_distance < 0.6:  # Adjust the threshold as needed
                    clusters[cluster_label].append((file_path, os.path.dirname(file_path)))
                    return True  # File added to a cluster
        # If the face does not match any existing cluster, create a new cluster
        new_cluster_label = max(clusters.keys()) + 1 if clusters else 0
        clusters[new_cluster_label] = [(file_path, os.path.dirname(file_path))]
        return True  # File added to a new cluster
    else:
        print("No face found in the provided image.")
        return False  # No face found in the image

new_file_path = "/content/group2/Elton_John23.jpg"  # Path to the new image
try:
  if add_file_to_clusters(new_file_path, clusters):
      print("New file added to clusters.")
  else:
      print("Failed to add new file to clusters.")
except:
  print("New File not uploaded")

<generator object _walk at 0x7d2ed9b4b3e0>

In [11]:
import shutil

# Replace 'folder_to_delete' with the name of the folder you want to delete
folder_to_delete = '/content/group2'

# Use shutil.rmtree to forcibly delete the folder and its contents
shutil.rmtree(folder_to_delete)

In [17]:
import os
import cv2
import numpy as np
import face_recognition
from sklearn.cluster import DBSCAN
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Read images from directories
def get_image_paths(directory_list):
    image_paths = []
    for directory in directory_list:
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.endswith(".jpg") or file.endswith(".png"):
                    image_paths.append(os.path.join(root, file))
    return image_paths

# Step 2 & 3: Detect faces and extract embeddings, modified to include directory information
def get_face_embeddings(image_paths):
    face_embeddings = []
    image_info = []  # Modified to store a tuple of image path and directory
    for image_path in image_paths:
        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.face_locations(image, model="cnn")
        if face_locations:
            # Assuming one face per image for simplicity
            face_encoding = face_recognition.face_encodings(image, known_face_locations=face_locations)[0]
            face_embeddings.append(face_encoding)
            image_info.append((image_path, os.path.dirname(image_path)))  # Store both image name and directory
    return face_embeddings, image_info

# # Step 4: Clustering embeddings
# def cluster_faces(face_embeddings):
#     clustering_model = DBSCAN(eps=0.5, min_samples=2, metric="euclidean").fit(face_embeddings)
#     labels = clustering_model.labels_
#     return labels

def cluster_faces_custom(face_embeddings, threshold=0.8):
    # Calculate the cosine similarity matrix
    cos_sim = cosine_similarity(face_embeddings)

    # Initialize labels array with -1 (indicating no cluster)
    labels = -1 * np.ones(len(face_embeddings), dtype=int)

    # Custom clustering based on threshold
    cluster_id = 0
    for i in range(len(face_embeddings)):
        if labels[i] == -1:  # If not already assigned to a cluster
            # Find embeddings with similarity above the threshold
            similar_indices = np.where(cos_sim[i] >= threshold)[0]
            for idx in similar_indices:
                labels[idx] = cluster_id  # Assign cluster ID
            cluster_id += 1  # Move to the next cluster ID for the next group

    return labels

# Step 5: Group image information by cluster, modified to handle directory information
def group_images_by_cluster(labels, image_info):
    clusters = defaultdict(list)
    for label, (image_name, directory) in zip(labels, image_info):
        clusters[label].append((image_name, directory))  # Store tuples of image name and directory in clusters
    return clusters

# Example usage
directory_list = ["/content/group1", "/content/group2"]  # Update these paths
image_paths = get_image_paths(directory_list)
face_embeddings, image_info = get_face_embeddings(image_paths)
# labels = cluster_faces(face_embeddings)
labels = cluster_faces_custom(face_embeddings, threshold=0.9)
clusters = group_images_by_cluster(labels, image_info)

# Print out clusters of images with similar faces along with their directories
for label, image_data in clusters.items():
    if label != -1:  # -1 is for outliers
        print(f"Cluster {label}:")
        for image_name, directory in image_data:
            print(f"{image_name} (Directory: {directory})")



Cluster 0:
/content/group1/Ben_Afflek11.jpg (Directory: /content/group1)
/content/group1/Ben_Afflek5.jpg (Directory: /content/group1)
/content/group1/Ben_Afflek9.jpg (Directory: /content/group1)
/content/group1/Ben_Afflek6.jpg (Directory: /content/group1)
/content/group1/Ben_Afflek7.jpg (Directory: /content/group1)
/content/group1/Ben_Afflek2.jpg (Directory: /content/group1)
/content/group1/Ben_Afflek10.jpg (Directory: /content/group1)
/content/group1/Ben_Afflek3.jpg (Directory: /content/group1)
/content/group1/Ben_Afflek1.jpg (Directory: /content/group1)
/content/group1/Ben_Afflek13.jpg (Directory: /content/group1)
/content/group1/Ben_Afflek4.jpg (Directory: /content/group1)
/content/group1/Ben_Afflek12.jpg (Directory: /content/group1)
/content/group2/Ben_Afflek14.jpg (Directory: /content/group2)
Cluster 1:
/content/group1/Jerry_Seinfield46.jpg (Directory: /content/group1)
/content/group1/Jerry_Seinfield32.jpg (Directory: /content/group1)
/content/group1/Jerry_Seinfield45.jpg (Directo

In [23]:
import os
import numpy as np
import face_recognition
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Read images from directories
def get_image_paths(directory_list):
    image_paths = []
    for directory in directory_list:
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.endswith(".jpg") or file.endswith(".png"):
                    image_paths.append(os.path.join(root, file))
    return image_paths

# Step 2 & 3: Detect faces and extract embeddings, modified for multiple faces per image
def get_face_embeddings(image_paths):
    face_embeddings = []
    image_info = []  # Modified to store tuples of image path, directory, and face index
    for image_path in image_paths:
        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.face_locations(image, model="cnn")
        for index, face_location in enumerate(face_locations):
            face_encoding = face_recognition.face_encodings(image, known_face_locations=[face_location])[0]
            face_embeddings.append(face_encoding)
            image_info.append((image_path, os.path.dirname(image_path), index))  # Include face index
    return face_embeddings, image_info

# Custom Clustering based on Cosine Similarity Threshold
def cluster_faces_custom(face_embeddings, threshold=0.8):
    # Calculate the cosine similarity matrix
    cos_sim = cosine_similarity(face_embeddings)

    # Initialize labels array with -1 (indicating no cluster)
    labels = -1 * np.ones(len(face_embeddings), dtype=int)

    # Custom clustering based on threshold
    cluster_id = 0
    for i in range(len(face_embeddings)):
        if labels[i] == -1:  # If not already assigned to a cluster
            similar_indices = np.where(cos_sim[i] >= threshold)[0]
            for idx in similar_indices:
                labels[idx] = cluster_id  # Assign cluster ID
            cluster_id += 1  # Increment for the next cluster

    return labels

# Step 5: Group image information by cluster, modified for multiple faces per image
def group_images_by_cluster(labels, image_info):
    clusters = defaultdict(list)
    for label, info in zip(labels, image_info):
        image_name, directory, face_index = info
        clusters[label].append(f"{image_name} [Face {face_index}] (Directory: {directory})")
    return clusters

# Example usage
directory_list = ["/content/g", "/content/group2"]  # Update these paths
image_paths = get_image_paths(directory_list)
face_embeddings, image_info = get_face_embeddings(image_paths)
labels = cluster_faces_custom(face_embeddings, threshold=0.95)
clusters = group_images_by_cluster(labels, image_info)

# Print out clusters of images with similar faces along with their directories
for label, images in clusters.items():
    if label != -1:  # -1 is for outliers
        print(f"Cluster {label}:")
        for data in images:
            print(data)


Cluster 0:
/content/g/2_2.jpg [Face 0] (Directory: /content/g)
/content/g/2_1.jpg [Face 0] (Directory: /content/g)
Cluster 1:
/content/g/2_2.jpg [Face 1] (Directory: /content/g)
/content/g/2_1.jpg [Face 1] (Directory: /content/g)
Cluster 2:
/content/g/2_2.jpg [Face 2] (Directory: /content/g)
/content/g/2_1.jpg [Face 2] (Directory: /content/g)
Cluster 3:
/content/g/2_2.jpg [Face 3] (Directory: /content/g)
Cluster 4:
/content/g/2-1_1.jpg [Face 0] (Directory: /content/g)
/content/g/2-1_2.jpg [Face 2] (Directory: /content/g)
Cluster 5:
/content/g/2-1_1.jpg [Face 1] (Directory: /content/g)
/content/g/2-1_2.jpg [Face 0] (Directory: /content/g)
Cluster 6:
/content/g/2-1_1.jpg [Face 2] (Directory: /content/g)
/content/g/2-1_2.jpg [Face 1] (Directory: /content/g)
Cluster 7:
/content/g/3_3.jpg [Face 0] (Directory: /content/g)
/content/g/3_1.jpg [Face 2] (Directory: /content/g)
/content/g/3_2.jpg [Face 0] (Directory: /content/g)
Cluster 8:
/content/g/3_3.jpg [Face 1] (Directory: /content/g)
/cont